In [14]:
import numpy as np
import pandas as pd
import math
import random
import time
from statistics import mean
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Definición de Dataset Inicial

In [12]:
x = [
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1],
    #A partir de aqui es relleno para el mini-batch
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1]
]
y = [1.03,-1.44,4.53,2.24,13.27,5.62,21.53,#A partir de aqui es relleno para el mini-batch
     1.03,-1.44,4.53,2.24,13.27,5.62,21.53]

#Definición de métodos utilizados

In [28]:
def signo(x):
  if x>=0:
    return 1
  else:
    return -1
def predict(modelo,x):
  if type(x[0]) == list:
    pred_v = []
    for x_it in x:
      pred = modelo[len(modelo)-1]
      for i in range(len(modelo)-1):
        pred+= modelo[i]*x_it[i]
      pred_v.append(pred)
    return pred_v
  else:
    pred = modelo[len(modelo)-1]
    for i in range(len(modelo)-1):
      pred+= modelo[i]*x[i]

    return pred

def MAE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  abs_v = [abs(ele) for ele in errs]

  return mean(abs_v)
def MSE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  squ_v = [(ele)**2 for ele in errs]

  return mean(squ_v)

def MSE_GRAD(modelo,x,errs,w_it,l_rate,reg_term,Lasso,Ridge):
  if w_it < len(modelo)-1:
        #len(modelo)-1 porque el último peso es el sesgo y se ajusta
        #de forma distinta
        #Aqui se ajustan todos los pesos menos el bias

        #Cálculo del Σ((Ŷ-Y)*Xji)
        for er in range(len(errs)):
          errs[er]= errs[er]*x[er][w_it]
        if Lasso:
          modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)+reg_term*signo(modelo[w_it])
        elif Ridge:
          modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)+2*reg_term*modelo[w_it]
        else:
          modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)

  else:
    #Aqui se ajusta el bias
    modelo[w_it] = modelo[w_it] -1*(2*l_rate*mean(errs) )

def MAE_GRAD(modelo,x,errs,w_it,l_rate,reg_term,Lasso,Ridge):
  if w_it < len(modelo)-1:
        for er in range(len(errs)):
          errs[er]= signo(errs[er])*x[er][w_it]
        if Lasso:
          modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)+reg_term*signo(modelo[w_it])
        elif Ridge:
          modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)+2*reg_term*modelo[w_it]
        else:
          modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)
        #Aqui se ajusta el bias
  else:
    errs = [signo(x) for x in errs]
    modelo[w_it] = modelo[w_it] -1*(l_rate*mean(errs) )

def generate_mini_batch(x, y, batch_percentage):
  tam_mini_batch = math.trunc(len(x)*batch_percentage)
  mini_batch_x = []
  mini_batch_y = []
  x_copy = x.copy()
  y_copy = y.copy()

  while len(x_copy)>0:
    it_mini_batchx = []
    it_mini_batchy = []
    while len(it_mini_batchx)<tam_mini_batch:
      index = random.randint(0,len(x_copy)-1)
      it_mini_batchx.append(x_copy[index])
      it_mini_batchy.append(y_copy[index])
      x_copy.pop(index)
      y_copy.pop(index)
    mini_batch_x.append(it_mini_batchx)
    mini_batch_y.append(it_mini_batchy)

  return mini_batch_x,mini_batch_y

def linear_regression(modelo,x,y,n_it,l_rate,reg_term,func_ajuste, Lasso=False, Ridge=False, minibatch_mode=False, batch_percentage=None):
  for _ in range(n_it):
    if not minibatch_mode:
      errs = []
      pred = predict(modelo,x)
      errs = [e1 - e2 for e1, e2 in zip(pred,y)]

      for w_it in range(len(modelo)):
        errs_it = errs.copy()
        func_ajuste(modelo,x,errs_it,w_it,l_rate,reg_term,Lasso,Ridge)
    else:
      batch_x, batch_y = generate_mini_batch(x, y, batch_percentage)
      for a, b in zip(batch_x, batch_y):
        errs = []
        pred = predict(modelo,a)
        errs = [e1 - e2 for e1, e2 in zip(pred,b)]

        for w_it in range(len(modelo)):
          errs_it = errs.copy()
          func_ajuste(modelo,a,errs_it,w_it,l_rate,reg_term,Lasso,Ridge)
  return modelo

#Prueba de los algoritmos de descenso del gradiente tanto con MAE como con MSE.

In [21]:
modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
start = time.time()
linear_regression(modelo,x,y,20000,0.001,0,MSE_GRAD)
tiempo = (time.time()-start)*1000
print('Error final -> ',MSE(modelo,x,y))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
start = time.time()
linear_regression(modelo,x,y,20000,0.001,0, MAE_GRAD)
tiempo = (time.time()-start)*1000
print('Error final -> ',MAE(modelo,x,y))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

Ajuste con MSE
Error inicial ->  54.367314285714286
Error final ->  0.008417023656379178
Tiempo de ejecución -> 3177.053689956665 ms
[6.007147684087246, -2.489069952946704, 0.06976974256473045]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.05959183673453323
Tiempo de ejecución -> 1252.2168159484863 ms
[6.008285714285928, -2.4745714285724416, -0.02685714285709419]


#Prueba de los algoritmos de descenso del gradiente tanto con MAE como con MSE comparación usando el modo batch y mini batch.

In [22]:
# Ajuste usando batch

print("Ajuste usando batch")

modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
start = time.time()
linear_regression(modelo,x,y,20000,0.001,0,MSE_GRAD,False,False, False)
tiempo = (time.time()-start)*1000
print('Error final -> ',MSE(modelo,x,y))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
start = time.time()
linear_regression(modelo,x,y,20000,0.001, 0, MAE_GRAD, False, False, False)
tiempo = (time.time()-start)*1000
print('Error final -> ',MAE(modelo,x,y))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

# Ajuste usando minibatch

print("\nAjuste usando minibatch")
modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
start = time.time()
linear_regression(modelo,x,y,20000,0.001,0,MSE_GRAD,False,False, True, 0.2)
tiempo = (time.time()-start)*1000
print('Error final -> ',MSE(modelo,x,y))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
start = time.time()
linear_regression(modelo,x,y,20000,0.001,0,MAE_GRAD,False,False, True, 0.2)
tiempo = (time.time()-start)*1000
print('Error final -> ',MAE(modelo,x,y))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

Ajuste usando batch
Ajuste con MSE
Error inicial ->  54.367314285714286
Error final ->  0.008417023656379178
Tiempo de ejecución -> 4216.405868530273 ms
[6.007147684087246, -2.489069952946704, 0.06976974256473045]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.05959183673453323
Tiempo de ejecución -> 1283.5075855255127 ms
[6.008285714285928, -2.4745714285724416, -0.02685714285709419]

Ajuste usando minibatch
Ajuste con MSE
Error inicial ->  54.367314285714286
Error final ->  0.007877147202557798
Tiempo de ejecución -> 9008.007049560547 ms
[6.024048135789265, -2.475020491140256, -0.01533394853407971]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.06928571428621066
Tiempo de ejecución -> 5638.964891433716 ms
[6.010000000000329, -2.4659999999998123, -0.0290000000000009]


#Implementación de términos de regularización L1 y L2; y su prueba con el conjunto *diabetes*

**Importante: Para que estas celdas funcionen se debe de mover el archivo diabetes.tab.txt a la carpeta del cuaderno y cambiar la extensión de .txt a .csv, por lo que el nuevo nombre del dataset sería diabetes.tab.csv**

Transformación del dataset a arrays para que el regresor pueda trabajar con él

In [29]:
data = pd.read_csv('diabetes.tab.csv',delimiter='\t')
def from_dataset_to_arrays(data):
  Y = data['Y']
  y = list(Y.values)
  data = data.drop(columns='Y')
  x = list(data.values)
  for a in range(len(x)):
    b = list(x[a])
    x[a] = b

  return x,y
x,y = from_dataset_to_arrays(data)

Se definen las funciones de error lasso y ridge

In [30]:
def lasso_error(modelo,x,y,reg_term,error_func):

    reg_error = error_func(modelo,x,y)
    L1=0
    for weight in range(len(modelo)-1):
        L1 += abs(modelo[weight])

    return reg_error + L1*reg_term

def ridge_error(modelo,x,y,reg_term,error_func):

    reg_error = error_func(modelo,x,y)
    L2=0
    for weight in range(len(modelo)-1):
        L2 += (modelo[weight])**2

    return reg_error + L2*reg_term

In [34]:
print('Ajuste lasso y ridge con conjunto de datos diabetes')

reg_term = 0.0001

print('\n---Ajuste lasso----')

modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('Ajuste con MSE')
print('Error inicial -> ',lasso_error(modelo,x,y,reg_term,MSE))
start = time.time()
linear_regression(modelo,x,y,2000,0.000001,reg_term, MSE_GRAD, True, False, False)
tiempo = (time.time()-start)*1000
print('Error final -> ',lasso_error(modelo,x,y,reg_term,MSE))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)


modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('\nAjuste con MAE')
print('Error inicial -> ',lasso_error(modelo,x,y,reg_term,MAE))
start = time.time()
linear_regression(modelo,x,y,2000,0.0001,reg_term,MAE_GRAD,True,False, False)
tiempo = (time.time()-start)*1000
print('Error final -> ',lasso_error(modelo,x,y,reg_term,MAE))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

print('\n---Ajuste ridge:---')

modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('Ajuste con MSE')
print('Error inicial -> ',ridge_error(modelo,x,y,reg_term,MSE))
start = time.time()
linear_regression(modelo,x,y,2000,0.000001,reg_term, MSE_GRAD,False, True, False)
tiempo = (time.time()-start)*1000
print('Error final -> ',ridge_error(modelo,x,y,reg_term,MSE))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)


modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('\nAjuste con MAE')
print('Error inicial -> ',ridge_error(modelo,x,y,reg_term,MAE))
start = time.time()
linear_regression(modelo,x,y,2000,0.0001,reg_term,MAE_GRAD,False, True, False)
tiempo = (time.time()-start)*1000
print('Error final -> ',ridge_error(modelo,x,y,reg_term,MAE))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(modelo)

Ajuste lasso y ridge con conjunto de datos diabetes

---Ajuste lasso----
Ajuste con MSE
Error inicial ->  155161.97646399765
Error final ->  4230.19390077608
Tiempo de ejecución -> 13684.542655944824 ms
[0.6692541784614949, 1.1768742124931129, 1.486988299536667, 1.1222994355327067, -0.2381551114877626, 0.3350863967192326, -0.642883121666363, 1.2665801559011485, 1.2206187757339493, 0.9895538407857866]

Ajuste con MAE
Error inicial ->  382.9561796380091
Error final ->  56.71728620076333
Tiempo de ejecución -> 10840.282678604126 ms
[0.7553592760180787, 1.1830407239818685, 1.2502112217194492, 0.9896000271493072, -0.38739140271490324, 0.4488893212669443, -0.211380090497738, 1.2364979638008782, 1.1961924376470796, 0.989307239818992]

---Ajuste ridge:---
Ajuste con MSE
Error inicial ->  155161.97646399765
Error final ->  4140.954288347782
Tiempo de ejecución -> 11937.187671661377 ms
[0.6938768935248033, 1.4595816237536823, 1.7681837002592886, 1.1367533513474015, -0.2271302183619681, 0.2298361

#Prueba utilizando un regresor de la librería *scikit-learn*

In [27]:
#Regresión lineal optimizando con el error cuadrático
reg = linear_model.LinearRegression()

start = time.time()
reg.fit(x, y)
tiempo = (time.time()-start)*1000

prediction = reg.predict(x)

print('Ajuste con MSE')
print('Error cuadrático -> ',mean_squared_error(y, prediction))
print('Error absoluto -> ', mean_absolute_error(y, prediction))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(f'[{reg.coef_} {reg.intercept_}]')

#Regresión lineal optimizando con el error ridge
reg = linear_model.Ridge()

start = time.time()
reg.fit(x, y)
tiempo = (time.time()-start)*1000

prediction = reg.predict(x)

print('\nAjuste con Ridge')
print('Error cuadrático -> ',mean_squared_error(y, prediction))
print('Error absoluto -> ', mean_absolute_error(y, prediction))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(f'[{reg.coef_} {reg.intercept_}]')

#Regresión lineal optimizando con el error lasso
reg = linear_model.Lasso()

start = time.time()
reg.fit(x, y)
tiempo = (time.time()-start)*1000

prediction = reg.predict(x)

print('\nAjuste con Lasso')
print('Error cuadrático -> ',mean_squared_error(y, prediction))
print('Error absoluto -> ', mean_absolute_error(y, prediction))
print(f'Tiempo de ejecución -> {tiempo} ms')
print(f'[{reg.coef_} {reg.intercept_}]')

Ajuste con MSE
Error cuadrático ->  2859.6963475867506
Error absoluto ->  43.27745202531506
Tiempo de ejecución -> 6.994724273681641 ms
[[-3.63612242e-02 -2.28596481e+01  5.60296209e+00  1.11680799e+00
 -1.08999633e+00  7.46450456e-01  3.72004715e-01  6.53383194e+00
  6.84831250e+01  2.80116989e-01] -334.5671385187875]

Ajuste con Ridge
Error cuadrático ->  2860.471596894782
Error absoluto ->  43.29169564350627
Tiempo de ejecución -> 9.291410446166992 ms
[[-3.28523969e-02 -2.26070454e+01  5.64040523e+00  1.11899757e+00
 -9.14673484e-01  5.84909825e-01  1.77885238e-01  6.25044178e+00
  6.31790809e+01  2.87766903e-01] -316.07711860428856]

Ajuste con Lasso
Error cuadrático ->  2901.5847284544093
Error absoluto ->  43.79459928373489
Tiempo de ejecución -> 14.918804168701172 ms
[[-1.90267115e-02 -1.74770497e+01  5.84243057e+00  1.09154258e+00
  1.56132396e-01 -3.15203518e-01 -1.18775171e+00  1.62432746e-01
  3.42247151e+01  3.29732281e-01] -202.30265547056504]
